<a href="https://colab.research.google.com/github/NIKKS1234/EMOTION_TO_EMOJI/blob/main/MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing all the libraries required for the program to work
import numpy as np
import random 
import matplotlib.pyplot as plt
import os
import glob as gb
import cv2
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,ZeroPadding2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
from keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.models import load_model

In [ ]:
picture_size = 48
folder_path = "C:/Users/AMARTY JHA/3D Objects/data"
output = r"C:\Users\AMARTY JHA\3D Objects"

In [ ]:
#showing sample dataset
expression = 'happy'

plt.figure(figsize= (12,12))
for i in range(1, 10, 1):
    plt.subplot(3,3,i)
    img = load_img(folder_path+"/train/"+expression+"/"+
                  os.listdir(folder_path + "/train/" + expression)[i], target_size=(picture_size, picture_size))
    plt.imshow(img)   
plt.show()

In [ ]:
# importing the training and testing data "FER 2013"
TRAIN_DIR = r"C:\Users\AMARTY JHA\3D Objects\data\train"
TEST_DIR = r"C:\Users\AMARTY JHA\3D Objects\data\test"
#batch_size is number of training examples utilized in one iterartion so here we are defining it is 128
BATCH_SIZE=128
# the RGB image has pixel range from 0-255 which is very high for our code so we are rescaling it to 0-1 by dividing it with 255 
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
#defining the output class variables
class_names = ['anger','disgusted','fearful','happy','neutral','sad','surprised']

In [ ]:
#data augmentation step
train_set = train_datagen.flow_from_directory(TRAIN_DIR,
                                                 target_size = (48, 48),
                                                 batch_size = 64,
                                                 class_mode = 'categorical',  #"categorical" will be 2D one-hot encoded labels
                                                 color_mode= "grayscale",
                                                 )

test_set = val_datagen.flow_from_directory(TEST_DIR,
                                            target_size = (48, 48),
                                            batch_size = 64,
                                            class_mode = 'categorical',
                                           color_mode= "grayscale",
                                            )

In [ ]:
#this is our customized cnn architecture 
emotion_model=Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(2,2))
emotion_model.add(Dropout(.25))

emotion_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(2,2))

emotion_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(2,2))
emotion_model.add(Dropout(.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(.5))
emotion_model.add(Dense(7, activation='softmax'))#using softmax to get relative probabilities of the seven emotions possible

In [ ]:
#this is vgg16 architetcture which is a pre-trained cnn model
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(48,48,1)))
model.add(Convolution2D(64, kernel_size=(3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, kernel_size=(3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, kernel_size=(3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, kernel_size=(3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, kernel_size=(3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, kernel_size=(3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))


model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512,kernel_size=(3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, kernel_size=(3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [ ]:
#compiling our model with loss , optimizer and metrics
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

#to get flow of model, uncomment below line
# model.summary()

In [ ]:
#training model
history = model.fit_generator(
train_set,
steps_per_epoch=28709//64,#448
epochs=25,
validation_data=test_set,
validation_steps=7178//64)
#this step can be skipped by directly downloading the "research50.h5" or "model.h5" file which has our saved model and directly running the next cell

In [ ]:
#saving weights of trained model
model.save('research50.h5')
#uncomment below if you want the vgg16 model 
#model.save('model.h5') 
print ('done')

In [ ]:
# rather than training the entire model which takes time , uncomment the line below to directly use the saved pre-trained model
#model = load_model(r"C:\Users\itsdi\research50.h5")
#uncomment below if you want vgg16 model
#model=load_model(r"C:\Users\itsdi\model.h5")
#in the line above in the load_model copy the path of file which has the saved model

In [ ]:
#evaluating the accuracy on the test data
model.evaluate(test_set)